<a href="https://colab.research.google.com/github/envomp/2020-Text-Mining/blob/master/week3/word_association_mining_syntacmatic_relations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_excel(r'2.xlsx')
